In [2]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import textwrap
import random
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from seqeval.metrics.sequence_labeling import get_entities

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("shibing624/bert4ner-base-chinese")
model = AutoModelForTokenClassification.from_pretrained("shibing624/bert4ner-base-chinese")
label_list = ['I-ORG', 'B-LOC', 'O', 'B-ORG', 'I-LOC', 'I-PER', 'B-TIME', 'I-TIME', 'B-PER']

# 全局变量
rows = 2
cols = 3
margin = 10
border_thickness = 0

show_grid = True

def get_entity(sentence):
    tokens = tokenizer.tokenize(sentence)
    inputs = tokenizer.encode(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(inputs).logits
    predictions = torch.argmax(outputs, dim=2)
    char_tags = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())][1:-1]

    pred_labels = [i[1] for i in char_tags]
    entities = []
    line_entities = get_entities(pred_labels)
    for i in line_entities:
        word = sentence[i[1]: i[2] + 1]
        entity_type = i[0]
        entities.append((word, entity_type))

    return entities


def create_rectangular_grid(img, border_thickness):
    # 计算每个矩形的大小
    rect_width = (img.shape[1] - (cols + 1) * margin) // cols
    rect_height = (img.shape[0] - (rows + 1) * margin) // rows

    # 创建一个与原始图像相同大小的空白图像
    grid_img = np.ones_like(img)*255

    # 绘制矩形
    if border_thickness > 0:
        for row in range(rows):
            for col in range(cols):
                x = margin + col * (rect_width + margin)
                y = margin + row * (rect_height + margin)
                cv2.rectangle(grid_img, (x, y), (x + rect_width, y + rect_height), (0,0,0), border_thickness)

    return grid_img

def get_random_rectangle(img):
    # 计算每个矩形的大小
    rect_width = (img.shape[1] - (cols + 1) * margin) // cols
    rect_height = (img.shape[0] - (rows + 1) * margin) // rows

    # 随机选择一个行和列
    row = random.randint(0, rows - 1)
    col = random.randint(0, cols - 1)

    # 计算矩形的坐标
    x = margin + col * (rect_width + margin)
    y = margin + row * (rect_height + margin)

    return x, y, x + rect_width, y + rect_height

def draw_text_in_rectangle(img, text, top_left, bottom_right, font_path, font_size):
    # 将OpenCV图像转换为PIL图像
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # 创建一个可以在图像上绘制的对象
    draw = ImageDraw.Draw(pil_img)

    # 加载字体
    font = ImageFont.truetype(font_path, font_size)

    # 计算矩形的宽度和高度
    rect_width = bottom_right[0] - top_left[0]
    rect_height = bottom_right[1] - top_left[1]

    # 计算单个字符的宽度和高度
    char_width, char_height = draw.textsize('测试', font=font)

    # 计算每行可以容纳的字符数
    chars_per_line = rect_width // char_width

    # 使用textwrap库将文本分割成多行
    lines = textwrap.wrap(text, width=2 * chars_per_line)

    # 在每一行中绘制文本
    y = top_left[1]
    for line in lines:
        if y + char_height <= bottom_right[1]:  # 确保文本不会超出矩形的底部
            draw.text((top_left[0], y), line, font=font, fill=(0, 0, 0, 0))
            y += char_height

    # 将PIL图像转换回OpenCV图像
    img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

    return img




In [3]:
# 创建一个示例图像
img = np.ones((800, 600, 3), np.uint8)  # 创建一个白色背景图像

# 用户输入的文本段落数
num_paragraphs = int(input("请输入需要排版的文字段落数："))

# 用户输入的文本段落
paragraphs = []
entities_list = []
for i in range(num_paragraphs):
    paragraph = input(f"请输入第{i+1}段文本：")
    entities = get_entity(paragraph)
    entity_text = [entity for entity in entities]
    paragraphs.append(paragraph)  # 将原始段落添加到列表中
    entities_list.extend(entity_text)  # 将实体添加到列表中

# 已经使用过的矩形
used_rectangles = []

# 类别词显示开关
show_original = True
show_LOC = True
show_PER = True
show_ORG = True
show_TIME = True

# 字体大小
original_font_size = 15
entity_font_size = 60

image_counter = 0

# 在主循环中使用这个函数
while True:
    # 创建矩形矩阵
    grid_img = create_rectangular_grid(img, border_thickness) if show_grid else img.copy()

    # 对于每个文本段落
    for i, text in enumerate(paragraphs):
        # 获取一个随机矩形的坐标
        for _ in range(500):  # 最多尝试500次
            x1, y1, x2, y2 = get_random_rectangle(img)
            if (x1, y1, x2, y2) not in used_rectangles:
                used_rectangles.append((x1, y1, x2, y2))
                break

        # 在随机矩形中绘制文本
        if show_original:
            grid_img = draw_text_in_rectangle(grid_img, text, (x1, y1), (x2, y2), "C:\\Windows\\Fonts\\simsun.ttc", original_font_size)

    # 对于每个实体
    for i, entity in enumerate(entities_list):
        # 获取一个随机矩形的坐标
        for _ in range(100):  # 最多尝试100次
            x1, y1, x2, y2 = get_random_rectangle(img)
            if (x1, y1, x2, y2) not in used_rectangles:
                used_rectangles.append((x1, y1, x2, y2))
                break

        # 在随机矩形中绘制文本
        if (entity[1] == 'LOC' and show_LOC) or (entity[1] == 'PER' and show_PER) or (entity[1] == 'ORG' and show_ORG) or (entity[1] == 'TIME' and show_TIME):
            grid_img = draw_text_in_rectangle(grid_img, entity[0], (x1, y1), (x2, y2), "C:\\Windows\\Fonts\\simsun.ttc", entity_font_size)

    # 在窗口中显示图像
    cv2.imshow('Grid', grid_img)

    key = cv2.waitKey(0) & 0xFF

    if key == ord('8'):
        rows += 1
    elif key == ord('2'):
        rows = max(1, rows - 1)
    elif key == ord('4'):
        cols = max(1, cols - 1)
    elif key == ord('6'):
        cols += 1
    elif key == ord(' '):  # 如果按下空格键，切换网格的显示状态
        border_thickness = 0 if border_thickness == 1 else 1  # 切换边框厚度
    elif key == ord('h'):  # 如果按下H键，切换LOC的显示状态
        show_LOC = not show_LOC
    elif key == ord('j'):  # 如果按下J键，切换PER的显示状态
        show_PER = not show_PER
    elif key == ord('k'):  # 如果按下K键，切换ORG的显示状态
        show_ORG = not show_ORG
    elif key == ord('l'):  # 如果按下L键，切换TIME的显示状态
        show_TIME = not show_TIME
    elif key == ord('7'):  # 如果按下7键，减小原文字体大小
        original_font_size = max(1, original_font_size - 1)
    elif key == ord('9'):  # 如果按下9键，增大原文字体大小
        original_font_size += 1
    elif key == ord('1'):  # 如果按下1键，减小类别词字体大小
        entity_font_size = max(1, entity_font_size - 1)
    elif key == ord('3'):  # 如果按下3键，增大类别词字体大小
        entity_font_size += 1
    elif key == ord('5'):
        cv2.imwrite('image_{}.jpg'.format(image_counter), grid_img)
        image_counter += 1
    # 按下ESC键退出循环
    elif key == 27:
        break

cv2.destroyAllWindows()


请输入需要排版的文字段落数：1
请输入第1段文本：岑春皓就读于上海交通大学，喜欢周末去程及美术馆玩。


C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', fon

C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', fon

C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', fon

C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', fon

...

C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', fon

C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', fon

C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', fon

C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', font=font)
C:\Users\21122\AppData\Local\Temp\ipykernel_2792\2359342341.py:90: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  char_width, char_height = draw.textsize('测试', fon

ValueError: invalid width 0 (must be > 0)